In [38]:
import openai
import chromadb
from chromadb.utils import embedding_functions
import pandas as pd
import config

In [39]:
with open('data/recycling guide.txt', 'r') as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]
    
data = {'text': lines}
df = pd.DataFrame(data)

df.head()

,text
0,재활용품은 재질별로 분류하여 분리수거함 및 별도의 전용수거함으로 배출한다.
1,대형 폐가전제품은 무상 방문 수거를 신청한다. 소형 폐가전제품도 같이 수거 가능하다.
2,"대형폐기물은 배출스티커를 붙히거나 인터넷 신청을 통해 수거 요청을 한다. 또는, 재..."
3,음식물류폐기물는 RFID 기반 음식물 폐기장을 사용하거나 종량제봉투에 담아 배출한다...
4,"유해폐기물로 구분된 폐전건지, 형광등, 폐의약품은 전용수거함을 이용한다."


In [40]:
df.tail()

,text
243,식품보조제 콜라겐가루가 남았는데 어떻게 버려야 하나요? 일반쓰레기로 버려야 하나요 ...
244,섬유유연제가 걸쭉하니 굳어가네요. 남은 양이 꽤 많은데 하수구에 버리자니 막힐 것 ...
245,다 쓴 프린터잉크 (페카트리지)는 어떻게 배출하나요?? 저희 읍에서는 종량제봉투에 ...
246,최근 아이스팩이 심각한 수질오염을 일으킬 수 있다는 얘기를 접했습니다. 단순히 종량...
247,"택배 올 때 비닐로 된 포장재가 있습니다. 뽁뽁이 말고요~ 연두빛 비닐인데, 산화 ..."


In [41]:
num_rows = df.shape[0]
print("DataFrame의 줄 개수:", num_rows)

DataFrame의 줄 개수: 248


In [42]:
def text_embedding(text):
    response = openai.Embedding.create(model="text-embedding-ada-002", input = text)
    return response["data"][0]["embedding"]
 
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = config.API['chatgpt'],
                model_name = "text-embedding-ada-002"
            )

#client = chromadb.Client()
client = chromadb.PersistentClient(path="DB")
collection = client.get_or_create_collection("recycling",embedding_function=openai_ef)

docs = df["text"].tolist()
ids = [str(x) for x in df.index.tolist()]

collection.add(
    documents = docs,
    ids = ids
)

In [43]:
vector = text_embedding("자전거는 어떻게 버려?")

results = collection.query(    
    query_embeddings = vector,
    n_results = 5,
    include = ["documents"]
)

res = "\n".join(str(item) for item in results['documents'][0])

print(res)

대형폐기물인 자전거는 배출스티커를 붙히거나 인터넷 신청을 통해 수거 요청을 한다. 또는, 재활용센터에 방문해 처리할 수 있다.
다 사용한 부탄가스통은 구멍을 뚫어서 배출해야 하나요? 안전사고의 위험으로 가급적 노즐을 누르거나 구멍을 뚫어 가스를 제거한 후 캔류로 배출합니다.
휴대폰배터리가 많아서 버릴려고하는데 어떻게 버려야할까요? 휴대폰과 스마트폰은 전기전자제품 재활용대상에 해당되며 소형가전제품 배출방법에 따라 배출하시면 됩니다.배터리는 휴대폰의 부속품으로 휴대폰과 같은 방법으로 배출하시면 되는데요 대형 전기전자제품은 한개만 배출해도 무상수거(1599-0903)가 가능하지만, 소형가전제품은 5개이상 모아서 배출해야 무상수거가 가능합니다. 지역 주민센터나 아파트 등에 설치된 전기전자제품 배출함이 있다면 그곳에 배출하셔도 되고요. 서울시와 같이 소형가전제품을 직접 재활용하는 지역의 경우에는 재활용품에 함께 배출하셔도 됩니다.
완구류 등은 어떻게 배출하나요? 재질별로 분리 후 플라스틱류로 배출합니다. 단, 재질별로 분리가 어려운 대형 완구류(유모차, 유아용 그네, 유아용 자동차, 목마 등)는 대형폐기물로 신고 후 배출수수료를 납부하고 배출합니다.
세안할 때 거품 많이 내려고 쓰는 거품망은 분리수거를 어떻게 해야 하나요? 플라스틱인지 비닐인지... 그리고 안에 스펀지도 있어서 더 헷갈리네요. 거품망은 나일론과 스펀지 재질로 재활용이 되지 않습니다. 종량제봉투에 담아 배출하여 주시기 바랍니다.


In [44]:
sys_prompt = f'''
너는 재활용 전문가로 활동하고 있어.
사람들이 재활용 방법을 물어보면 너는 주어진 Context를 바탕으로 자세하게 답해줘야해.

Context: {res}
'''

prompt = '자전거는 어떻게 버려?'

messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": prompt}
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0
)
response_message = response["choices"][0]["message"]["content"]

print(response_message)

자전거는 대형폐기물에 해당되므로, 배출스티커를 붙히거나 인터넷을 통해 수거 요청을 할 수 있습니다. 또는, 가급적 재활용센터에 직접 방문하여 처리할 수도 있습니다. 이렇게 함으로써 환경을 보호하고 재활용을 촉진할 수 있습니다.
